# Conversational RAG

## Intro
* In many Q&A applications we want to allow user to have back-and-forth conversation, meaning that application needs some sort of "memory" of past questions and answers

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
MODEL_GPT = 'gpt-4o-mini'

## Connect with LLM

In [3]:
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(model=MODEL_GPT)

In [4]:
#!pip install langchain-community langchainhub langchain-chroma bs4

## Basic RAG without memory

In [5]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# loader = TextLoader("./data/be-good.txt")
loader = TextLoader("../../data/be-good.txt")

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [6]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [7]:
output = rag_chain.invoke({"input": "What is this article about?"})

In [8]:
output["answer"]

'The article discusses the principles of creating successful startups, emphasizing the importance of making something people want while not overly focusing on the business model initially. It suggests that a business can operate similarly to a charity, where good intentions and user satisfaction lead to success, using Craigslist as an example. The author reflects on the balance between benevolence and profitability in entrepreneurship.'

In [9]:
output = rag_chain.invoke({"input": "What was my previous question about?"})

In [10]:
output["answer"]

"I don't know."

## Adding memory

In [11]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [12]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [13]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is this article about?"

ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "What was my previous question about?"

ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Your previous question was about the content and focus of the article, asking what it is about.


In [14]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [15]:
conversational_rag_chain.invoke(
    {"input": "What is this article about?"},
    config={
        "configurable": {"session_id": "001"}
    },  # constructs a key "001" in `store`.
)["answer"]

'The article discusses the principles of creating successful businesses, emphasizing the importance of making something people want and not overly focusing on immediate profitability. It critiques the notion that well-intentioned projects automatically yield positive outcomes, using examples like Craigslist to illustrate a model that operates successfully without traditional profit-driven motives. Overall, it suggests a potential new approach to business that aligns more closely with charitable principles.'

In [16]:
conversational_rag_chain.invoke(
    {"input": "What was my previous question about?"},
    config={"configurable": {"session_id": "001"}},
)["answer"]

'Your previous question was about the content or main theme of the article.'

In [17]:
for message in store["001"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is this article about?

AI: The article discusses the principles of creating successful businesses, emphasizing the importance of making something people want and not overly focusing on immediate profitability. It critiques the notion that well-intentioned projects automatically yield positive outcomes, using examples like Craigslist to illustrate a model that operates successfully without traditional profit-driven motives. Overall, it suggests a potential new approach to business that aligns more closely with charitable principles.

User: What was my previous question about?

AI: Your previous question was about the content or main theme of the article.

